# Notebook 2: Tire Degradation Rules Implementation

This notebook focuses on implementing the rule set related to tire degradation for our F1 Strategy Engine. These rules are crucial for race strategy as they determine when a car should pit based on tire performance analysis.

## Overview of Degradation Rules

We'll implement three core rules:

1. **High Degradation Rate Pit Stop**
   - IF (DegradationRate > 0.15 AND TyreAge > 10)
   - THEN recommend priority pit stop
   - CONFIDENCE: 0.85

2. **Stint Extension for Low Degradation**
   - IF (DegradationRate < 0.08 AND TyreAge > 12 AND Position < 5)
   - THEN recommend extending current stint
   - CONFIDENCE: 0.75

3. **Early Degradation Warning**
   - IF (DegradationRate increases by more than 0.03 in 3 consecutive laps)
   - THEN recommend pit stop preparation
   - CONFIDENCE: 0.7

Each rule handles specific aspects of tire degradation strategy that F1 teams consider during races.

---

## 1. Importing necessary Libraries

In [ ]:
# Import standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import sys
sys.path.append(os.path.abspath('../'))  
from ML_tyre_pred.ML_utils import N02_model_tire_predictions as tdp

# Import Experta components
from experta import Rule, NOT, OR, AND, AS, MATCH, TEST, EXISTS
from experta import DefFacts, Fact, Field, KnowledgeEngine



####################### Import Custom Fact Classes ###################
import utils.N01_agent_setup as agent_setup
from utils.N01_agent_setup import (
    TelemetryFact,
    DegradationFact,
    RaceStatusFact,
    StrategyRecommendation,
    F1StrategyEngine,
    transform_tire_predictions,
    get_current_degradation,
    load_tire_predictions
)
 
# Configuring plots 

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_context("notebook", font_scale= 1.2)

print("Libraries and fact classes loaded successfully.")

---

## 2. Making variable analysis for specifying thresholds

### 2.1 Analyzing Degradation Data

In [ ]:
# Load and prepare tire degradation data
def load_degradation_data(file_path='../../outputs/week5/tire_degradation_fuel_adjusted.csv'):
    """
    Load real tire degradation data and prepare it for rule evaluation
    
    Args:
        file_path: Path to the degradation data CSV file
        
    Returns:
        DataFrame with processed tire degradation data
    """
    df = pd.read_csv(file_path)
    print(f"Successfully loaded data from {file_path}")
    
    # Convert float columns to integers where appropriate
    integer_columns = ['Position', 'TyreAge', 'DriverNumber', 'CompoundID', 'TeamID']
    for col in integer_columns:
        if col in df.columns:
            df[col] = df[col].astype(int)
    
    # Sort data for consistency
    df = df.sort_values(['DriverNumber', 'Stint', 'TyreAge'])
    
    # Calculate race lap by accumulating TyreAge across stints
    # First, get the maximum TyreAge for each completed stint
    max_age_by_stint = df.groupby(['DriverNumber', 'Stint'])['TyreAge'].max().reset_index()
    max_age_by_stint = max_age_by_stint.rename(columns={'TyreAge': 'StintLength'})
    
    # Create a lookup for previous stint lengths
    stint_lengths = {}
    for driver in df['DriverNumber'].unique():
        driver_stints = max_age_by_stint[max_age_by_stint['DriverNumber'] == driver]
        
        # Calculate cumulative stint lengths
        cumulative_lengths = [0]  # Start with 0 for the first stint
        for i in range(len(driver_stints) - 1):
            cumulative_lengths.append(
                cumulative_lengths[-1] + driver_stints.iloc[i]['StintLength']
            )
        
        # Store in dictionary
        stint_lengths[driver] = {
            stint: length for stint, length in zip(
                driver_stints['Stint'], cumulative_lengths
            )
        }
    
    # Function to calculate race lap
    def calculate_race_lap(row):
        driver = row['DriverNumber']
        stint = row['Stint']
        tyre_age = row['TyreAge']
        
        # Get the starting lap for this stint
        start_lap = stint_lengths.get(driver, {}).get(stint, 0)
        
        # Add current TyreAge to get race lap
        return start_lap + tyre_age
    
    # Apply function to calculate race lap
    df['RaceLap'] = df.apply(calculate_race_lap, axis=1)
    
    # For each driver-stint combination, calculate previous degradation rates
    # This will be useful for Rule 3 (Early Degradation Warning)
    def get_previous_rates(group, n=3):
        """Get previous n degradation rates for each row in the group"""
        rates = []
        for i in range(len(group)):
            if i < n:
                # Not enough previous data
                rates.append(group.iloc[:i+1]['DegradationRate'].tolist())
            else:
                # Get last n rates including current
                rates.append(group.iloc[i-n+1:i+1]['DegradationRate'].tolist())
        return rates
    
    # Apply function to each driver-stint group
    df['PreviousRates'] = df.groupby(['DriverNumber', 'Stint']).apply(
        lambda x: get_previous_rates(x)
    ).explode().tolist()
    
    return df



In [ ]:
# Load the data
degradation_data = load_degradation_data()

# Display sample of the data
print("Loaded degradation data:")
display(degradation_data.head())



---

### 2.2 Chossing thresholds for degradation

In [ ]:
# Analyze DegradationRate distribution to set appropriate thresholds
print("DegradationRate statistics:")
display(degradation_data['DegradationRate'].describe())

# Distribution of positive degradation values (actual degradation)
positive_deg = degradation_data[degradation_data['DegradationRate'] > 0]['DegradationRate']
print("\nStatistics for positive degradation values only:")
display(positive_deg.describe())

# Visualize the distribution
plt.figure(figsize=(10, 6))
plt.hist(positive_deg, bins=30, alpha=0.7)
plt.axvline(positive_deg.quantile(0.75), color='r', linestyle='--', 
           label=f'75th percentile: {positive_deg.quantile(0.75):.3f}')
plt.axvline(positive_deg.quantile(0.25), color='g', linestyle='--',
           label=f'25th percentile: {positive_deg.quantile(0.25):.3f}')
plt.xlabel('Degradation Rate (seconds/lap)')
plt.ylabel('Frequency')
plt.title('Distribution of Positive Degradation Rates')
plt.legend()
plt.tight_layout()
plt.show()

# Suggested thresholds based on data percentiles
high_degradation_threshold = positive_deg.quantile(0.75)  # 75th percentile
low_degradation_threshold = positive_deg.quantile(0.25)   # 25th percentile

print(f"\nSuggested thresholds based on data distribution:")
print(f"High Degradation Threshold: {high_degradation_threshold:.3f} seconds/lap")
print(f"Low Degradation Threshold: {low_degradation_threshold:.3f} seconds/lap")

---

### 2.3 Plotting degradation for sample drivers for putting thresholds

In [ ]:
# Plot degradation for a sample driver across race laps
sample_driver = degradation_data['DriverNumber'].unique()[0]
driver_data = degradation_data[degradation_data['DriverNumber'] == sample_driver]

# Visualize degradation with the correct thresholds and limited y-axis
plt.figure(figsize=(12, 6))
plt.plot(driver_data['RaceLap'], driver_data['DegradationRate'], 'o-', linewidth=2)
plt.axhline(y=0.3, color='r', linestyle='--', label='High Degradation Threshold (0.3)')
plt.axhline(y=0.15, color='g', linestyle='--', label='Low Degradation Threshold (0.15)')

# Mark stint changes
stint_changes = []
for i in range(1, len(driver_data)):
    if driver_data.iloc[i]['Stint'] != driver_data.iloc[i-1]['Stint']:
        stint_changes.append(driver_data.iloc[i]['RaceLap'])

for lap in stint_changes:
    plt.axvline(x=lap, color='k', linestyle='--', label='Pit Stop' if 'Pit Stop' not in plt.gca().get_legend_handles_labels()[1] else "")

# Stablish Y axis limits for not putting anything below -1 in the graphic. Important for seeing the real scale
plt.ylim(bottom=-1)

plt.xlabel('Race Lap')
plt.ylabel('Degradation Rate (seconds/lap)')
plt.title(f'Tire Degradation Profile for Driver {sample_driver}')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

---

## 3. Defining the Engine Class with all three degradation rules

---